<a href="https://colab.research.google.com/github/kimyougmin/TextClassifierApi/blob/main/TextClassifierApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1
!pip install gluonnlp==0.10.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch==1.0.0 torchvision==0.4.0
!pip install FastAPI -q
!pip install uvicorn -q
!pip install light-the-torch
!pip install nest_asyncio
!pip install fastapi nest-asyncio pyngrok uvicorn
!ngrok authtoken 2giutVPi7nEYUWd5I7wHHYZFupu_67WJ3pzANFoMXAaUVCfyK
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Using cached transformers-3.0.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.8.1rc1.tar.gz (97 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
ERROR: Could not find a version that satisfies the requirement torch==1.0.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 

In [ ]:
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda")

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1',return_dict=False) #return_dict=False
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
data_list = []
data_testList = []

def changeToNumber(x) :
  if x == "1.0" : return 1
  else : return 0
max_len = 64 # max seqence length
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

data = pd.read_csv("/content/Dataset.csv")
test = pd.read_csv("/content/testData.csv")
for content, clean, curse, conflictOfGeneration, insult, caricature in zip(data.content, data.clean, data.curse, data.conflictOfGeneration, data.insult, data.caricature):
  data = []
  data.append(str(content))
  if 1 == changeToNumber(str(clean)) :
      data.append('0')
  elif 1 == changeToNumber(str(curse)) :
      data.append('1')
  elif 1 == changeToNumber(str(conflictOfGeneration)) :
    data.append('2')
  elif 1 == changeToNumber(str(insult)) :
    data.append('3')
  else :
    data.append('4')
  data_list.append(data)

for content, clean, curse, conflictOfGeneration, insult, caricature in zip(test.content, test.clean, test.curse, test.conflictOfGeneration, test.insult, test.caricature):
  data = []
  data.append(str(content))
  if 1 == changeToNumber(str(clean)) :
      data.append('0')
  elif 1 == changeToNumber(str(curse)) :
      data.append('1')
  elif 1 == changeToNumber(str(conflictOfGeneration)) :
    data.append('2')
  elif 1 == changeToNumber(str(insult)) :
    data.append('3')
  else :
    data.append('4')
  data_testList.append(data)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# from sklearn.model_selection import train_test_split
# dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

tok = tokenizer.tokenize

data_train = BERTDataset(data_list, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(data_testList, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

#정의한 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #,  dr_rate=0.5
# Prepare optimizer and schedule (linear warmup and decay)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
#     model.eval()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

# torch.save(model.state_dict(), "/content/drive/MyDrive/model.pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/model.pt"));

predict_item = ''
category={'clean':0, 'curse': 1, 'conflictOfGeneration':2, 'insult':3, 'caricature':4};
# acc = [0,0,0,0,0]
# acc_tp = [0,0,0,0,0]
# acc_fp = [0,0,0,0,0]
# acc_fn = [0,0,0,0,0]
# acc_tn = [0,0,0,0,0]

# precision = [0,0,0,0,0]
# pre_tp = [0,0,0,0,0]
# pre_fp = [0,0,0,0,0]

# recell = [0,0,0,0,0]
# re_tp = [0,0,0,0,0]
# re_fn = [0,0,0,0,0]

# f1 = [0,0,0,0,0]
# def arrayCheck(array):
#   if array == '0' :
#     return 'clean'
#   elif array == '1':
#     return 'curse'
#   elif array == '2':
#     return 'conflictOfGeneration'
#   elif array == '3':
#     return 'insult'
#   elif array == '4':
#     return 'caricature'
# def precisionCheck(predicted, actual) :
#   if predicted == 'clean' and actual == 'clean':
#     pre_tp[0] += 1
#   elif predicted == 'clean' and actual != 'clean' :
#     pre_fp[0] += 1

#   if predicted == 'curse' and actual == 'curse':
#     pre_tp[1] += 1
#   elif predicted == 'curse' and actual != 'curse' :
#     pre_fp[1] += 1

#   if predicted == 'conflictOfGeneration' and actual == 'conflictOfGeneration':
#     pre_tp[2] += 1
#   elif predicted == 'conflictOfGeneration' and actual != 'conflictOfGeneration' :
#     pre_fp[2] += 1

#   if predicted == 'insult' and actual == 'insult':
#     pre_tp[3] += 1
#   elif predicted == 'insult' and actual != 'insult' :
#     pre_fp[3] += 1

#   if predicted == 'insult' and actual == 'insult':
#     pre_tp[4] += 1
#   elif predicted == 'insult' and actual != 'insult' :
#     pre_fp[4] += 1

# def recallCheck(predicted, actual):
#   if predicted == 'clean' and actual == 'clean':
#     re_tp[0] += 1
#   elif predicted != 'clean' and actual == 'clean' :
#     re_fn[0] += 1

#   if predicted == 'curse' and actual == 'curse':
#     re_tp[1] += 1
#   elif predicted != 'curse' and actual == 'curse' :
#     re_fn[1] += 1

#   if predicted == 'conflictOfGeneration' and actual == 'conflictOfGeneration':
#     re_tp[2] += 1
#   elif predicted != 'conflictOfGeneration' and actual == 'conflictOfGeneration' :
#     re_fn[2] += 1

#   if predicted == 'insult' and actual == 'insult':
#     re_tp[3] += 1
#   elif predicted != 'insult' and actual == 'insult' :
#     re_fn[3] += 1

#   if predicted == 'insult' and actual == 'insult':
#     re_tp[4] += 1
#   elif predicted != 'insult' and actual == 'insult' :
#     re_fn[4] += 1

# def accyracyCheck(predicted, actual):
#   if actual == 'clean' and predicted == 'clean' :
#     acc_tp[0] += 1
#   elif actual == 'clean' and predicted != 'clean' :
#     acc_fn[0] += 1
#   elif actual != 'clean' and predicted == 'clean' :
#     acc_fp[0] += 1
#   elif actual != 'clean' and predicted != 'clean' :
#     acc_tn[0] += 1

#   if actual == 'curse' and predicted == 'curse' :
#     acc_tp[1] += 1
#   elif actual == 'curse' and predicted != 'curse' :
#     acc_fn[1] += 1
#   elif actual != 'curse' and predicted == 'curse' :
#     acc_fp[1] += 1
#   elif actual != 'curse' and predicted != 'curse' :
#     acc_tn[1] += 1

#   if actual == 'conflictOfGeneration' and predicted == 'conflictOfGeneration' :
#     acc_tp[2] += 1
#   elif actual == 'conflictOfGeneration' and predicted != 'conflictOfGeneration' :
#     acc_fn[2] += 1
#   elif actual != 'conflictOfGeneration' and predicted == 'conflictOfGeneration' :
#     acc_fp[2] += 1
#   elif actual != 'conflictOfGeneration' and predicted != 'conflictOfGeneration' :
#     acc_tn[2] += 1

#   if actual == 'insult' and predicted == 'insult' :
#     acc_tp[3] += 1
#   elif actual == 'insult' and predicted != 'insult' :
#     acc_fn[3] += 1
#   elif actual != 'insult' and predicted == 'insult' :
#     acc_fp[3] += 1
#   elif actual != 'insult' and predicted != 'insult' :
#     acc_tn[3] += 1

#   if actual == 'caricature' and predicted == 'caricature' :
#     acc_tp[4] += 1
#   elif actual == 'caricature' and predicted != 'caricature' :
#     acc_fn[4] += 1
#   elif actual != 'caricature' and predicted == 'caricature' :
#     acc_fp[4] += 1
#   elif actual != 'caricature' and predicted != 'caricature' :
#     acc_tn[4] += 1

def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another =[data]
  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
  test_dataLoader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataLoader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length = valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)
    test_eval = []
    for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      test_eval.append(list(category.keys())[(np.argmax(logits))])
    return test_eval[0]
# for (e, index) in data_testList:
#   predict_item = predict(e)
#   index_check = arrayCheck(index)
#   accyracyCheck(predict_item, index_check)
#   precisionCheck(predict_item, index_check)
#   recallCheck(predict_item, index_check)
#   print(e, ': 예측 =>', predict_item)
# for i in range(len(acc_tp)):
#   acc[i] = (acc_tp[i]+acc_tn[i])/(acc_tp[i]+ acc_fp[i] + acc_fn[i] + acc_tn[i])
#   precision[i] = (pre_tp[i])/(pre_tp[i] + pre_fp[i])
#   recell[i] = (re_tp[i])/(re_tp[i] + re_fn[i])
#   f1[i] = 2* ((precision[i]*recell[i])/(precision[i]+recell[i]))
# print('Accuracy \n', acc)
# print('Precision \n', precision)
# print('recell \n', recell)
# print('f1-scour \n', f1)
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import json
import asyncio
import time
from starlette.requests import Request
from starlette.responses import Response
import nest_asyncio
from pydantic import BaseModel

class InputText(BaseModel):
  text: str

  def __len__(self):
        return len(self.text)

!export NGROK_AUTH_TOKEN=2giutVPi7nEYUWd5I7wHHYZFupu_67WJ3pzANFoMXAaUVCfyK
origins = [
    "http://localhost:8000/",
    "http://localhost:3000/",
    "http://localhost:3000",
    "https://localhost:8000/",
    "https://localhost:3000/",
    "https://localhost:3000",
]
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get("/predicta/{id}")
async def predicta(id):
    result = predict(id)
    print("text", id, "classification", result)
    return {"text": id, "classification": result}

@app.post("/items/", response_model=None)
async def create_item(item: InputText):
    result = predict(item.text)
    print("text", item.text, "classification", result)
    return {"text": item.text, "classification": result}

from fastapi import APIRouter, FastAPI

router = APIRouter()
app.include_router(router)


def start_sever():
    nest_asyncio.apply()
    uvicorn.run(app, port=8000)
if __name__ == "__main__":
    ngrok_tunnel = ngrok.connect(8000)
    print('공용 URL:', ngrok_tunnel.public_url)
    start_sever()

<ipython-input-6-4068361250f0>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/model.pt"));
INFO:     Started serve

공용 URL: https://4bbe-34-90-246-34.ngrok-free.app
INFO:     218.239.107.128:0 - "OPTIONS /items HTTP/1.1" 200 OK
INFO:     218.239.107.128:0 - "POST /items HTTP/1.1" 307 Temporary Redirect
INFO:     218.239.107.128:0 - "OPTIONS /items/ HTTP/1.1" 200 OK
text 테스트 메시지 classification clean
INFO:     218.239.107.128:0 - "POST /items/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2243]


# 새 섹션

In [ ]:
# from fastapi import FastAPI
# from fastapi.middleware.cors import CORSMiddleware
# import nest_asyncio
# from pyngrok import ngrok
# import uvicorn
# import json
# import asyncio
# import time
# from starlette.requests import Request
# from starlette.responses import Response

# !export NGROK_AUTH_TOKEN=2giutVPi7nEYUWd5I7wHHYZFupu_67WJ3pzANFoMXAaUVCfyK

# print(predict('씨발년아'))
# app = FastAPI()
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=['*'],
#     allow_credentials=True,
#     allow_methods=['*'],
#     allow_headers=['*'],
# )
# @app.get('/')
# async def root():
#     return {'hello': 'world'}

# @app.get("/predicta/{id}")
# async def predicta(id):
#     await time.sleep(1.0)
#     return {"valus": predict(id)}

# ngrok_tunnel = ngrok.connect(8000)
# print('공용 URL:', ngrok_tunnel.public_url)
# uvicorn.run(app, port=8000)

In [ ]:
class BERTDataset:
    INITIALIZE DATASET:
        INPUT: dataset, sentence index, label index, BERT tokenizer, vocabulary, max length, padding, pair option
        CREATE transform FUNCTION:
            Use BERT tokenizer to transform sentences based on max length, vocabulary, padding, and pair option

        FOR EACH item in dataset:
            Transform the sentence using the transform function and store it
            Extract and store the corresponding label

    METHOD: __getitem__ (GET ITEM):
        INPUT: index i
        RETURN: transformed sentence and its label at index i

    METHOD: __len__ (LENGTH):
        RETURN: the total number of labels in the dataset

'3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]'

# 새 섹션